In [4]:
import os, glob, time, sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC

In [5]:
os.makedirs("AU Housing Data", exist_ok=True)
Data_Path = os.path.join("AU Housing Data", "AU Real Estate Data.csv")
House_List = pd.read_csv(Data_Path)

In [6]:
class Scrape_Web:
    def __init__(self, URL):
        self.URl = URL
    
    def Scrape_Page(self):
        Browser = webdriver.Edge()
        Browser.get(self.URl)
        Browser.set_window_size(1280, 720)

        time.sleep(10)
        Source = BeautifulSoup(Browser.page_source, 'lxml')

        Areas = Source.find_all("div", class_="View__PropertySizeGroup-sc-1psmy31-1 iklnuy property-size-group")
        Locations = Source.find_all("h2", class_="residential-card__address-heading")
        Prices = Source.find_all("div", class_="residential-card__price")
        Bedrooms = Source.find_all("div", aria_label=lambda x: x and "bedrooms" in x) 
        Bathrooms = Source.find_all("div", aria_label=lambda x: x and "bathrooms" in x)

        House_List = []

        for Area, Location, Price, Bedroom, Bathroom in zip(Areas, Locations, Prices, Bedrooms, Bathrooms):
            Bedroom = Bedroom.find('p', class_="Text__Typography-sc-vzn7fr-0 QtGDW").text.strip() 
            Bathroom = Bathroom.find('p', class_="Text__Typography-sc-vzn7fr-0 QtGDW").text.strip() 
            Location = Location.find('span').get_text().strip()  
            Area = Area.get_text().strip()  
            Price = Price.find('span').text.strip()  

            House_List.append({
                'Area': Area,
                'Location': Location,
                'Price': Price,
                'Bedroom': Bedroom,
                'Bathroom': Bathroom
            })

        Browser.quit()  
        return House_List

def scrape_multiple_pages(self, Total_Pages):
    All_House_List = []

    for Page_Num in range(1, Total_Pages + 1):
        if Page_Num == 1:
            URLs = self.URL 
        else:
            URLs = f"{self.URL[:-1]}-{Page_Num}" 

        print(f"Scraping Page {Page_Num}: {URLs}")
    
        Houses = self.Scrape_Page(URLs)
        All_House_List.extend(Houses)

    return All_House_List   


In [7]:
class Data_Cleanning:
    def __init__(self, House_List):
        self.Dataset = House_List
        return self.Data_Cleaning()
    
    def Data_Cleaning(self):
        self.City_Mapping = {City: index for index, City in enumerate(self.Dataset["City"].unique())}  
        self.State_Mapping = {State: index for index, State in enumerate(self.Dataset["State"].unique())}
        self.Type_Mapping = {Type: index for index, Type in enumerate(self.Dataset["Type"].unique())}

        self.Dataset.loc[:, "City"] = self.Dataset["City"].map(self.City_Mapping)
        self.Dataset.loc[:, "State"] = self.Dataset["State"].map(self.State_Mapping)
        self.Dataset.loc[:, "Type"] = self.Dataset["Type"].map(self.Type_Mapping)
        
        self.Dataset = self.Dataset.dropna(subset=['Price', 'Bedrooms', 'Bathrooms', 'SqFt', 'City', 'State', 'Year Built', 'Type', 'Garage', 'Lot Area'])
        return self.Dataset.to_csv(os.path.join("AU Housing Data", "Cleaned - AU Real Estate Data.csv"), index=False)
    
    def Inverse_Mapping(self):  
        self.Inverse_City_Mapping = {index: City for City, index in self.City_Mapping.items()}  
        self.Inverse_State_Mapping = {index: State for State, index in self.State_Mapping.items()}   
        self.Inverse_Type_Mapping =  {index: Type for Type, index in self.Type_Mapping.items()} 


In [8]:
class SQLite3:
    def __init__(self, Dataset):
        self.House_List = Dataset
        
    def House_Price_Statistics(self):
        if 'AU Real Estate.db' not in glob.glob("*.db"):
            Connection = sqlite3.connect('AU Real Estate.db')
            self.House_List.to_sql('AU Real Estate', Connection, if_exists='replace', index=False)
            print("Database already created!")
            return Connection
        else: return sqlite3.connect('AU Real Estate.db')
    
    def Exc(self, Query):
        Connection = self.House_Price_Statistics()
        return pd.read_sql_query(Query, Connection)
    
Connection = SQLite3(House_List)

Query = """
select *
from "AU Real Estate"
"""
Connection.Exc(Query)

Database already created!


,Price,Bedrooms,Bathrooms,SqFt,City,State,Year Built,Type,Garage,Lot Area
0,982112,4,1,1561,Adelaide,QLD,2021,Townhouse,0,1357
1,571388,3,2,3735,Melbourne,VIC,1999,House,1,8397
2,866821,5,1,2032,Sydney,VIC,1976,Townhouse,0,3478
3,1230977,4,3,3861,Brisbane,SA,1978,House,0,7619
4,241787,4,2,3150,Perth,SA,1992,Apartment,0,8324
...,...,...,...,...,...,...,...,...,...,...
4995,682555,4,3,887,Adelaide,QLD,2012,Townhouse,0,4093
4996,1634457,3,1,1117,Brisbane,QLD,2020,Townhouse,1,5193
4997,1209978,4,1,992,Perth,QLD,2008,House,0,4318
4998,1976680,3,3,1825,Sydney,QLD,1957,House,0,6661
